In [1]:
from pyspark import SparkContext, SparkConf
from operator import add

conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


from pyspark.sql.types import *

In [2]:
df = sc.textFile('./moviedata/rating.csv')

In [3]:
column_data = df.map(lambda x: x.split(","))

In [13]:
movieid_rating = column_data.map(lambda x:(str(x[1]),(float(x[2]), 1)))

In [18]:
movieid_rating.filter(lambda x: x[0]=='452').count()

62

In [11]:
f = movieid_rating.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))
# ALTERNATIVE: 
# f = movieid_rating.aggregateByKey((0,0), 
#                                   (lambda x, y: (x[0]+y, x[1]+1)),
#                                   (lambda rdd1, rdd2: (rdd1[0]+rdd2[0], rdd1[1]+rdd2[1])))

In [12]:
f.take(5)

[('452', (165.0, 62)),
 ('1269', (39.0, 10)),
 ('1524', (34.0, 8)),
 ('1144', (8.0, 2)),
 ('818', (66.0, 24))]

In [21]:
movie_avgrating = f.map(lambda x:(x[0],x[1][0]/x[1][1]))

In [22]:
movie_avgrating.takeOrdered(10, key = lambda x: -x[1])

[('1467', 5.0),
 ('814', 5.0),
 ('1191', 5.0),
 ('1122', 5.0),
 ('1500', 5.0),
 ('1201', 5.0),
 ('1491', 5.0),
 ('1653', 5.0),
 ('1189', 5.0),
 ('1536', 5.0)]